In [1]:
import pandas as pd

df = pd.read_csv('Datos/Originales/Datos UX/page_views_2.csv', sep=';')
df["event_timestamp_"] = pd.to_datetime(df["event_timestamp_"])
df = df.sort_values(["user_id", "event_timestamp_"])
df.head()

,event_name,section,user_id,event_date_,event_timestamp_
0,page_view,quiz_work,040e1e30c9ed4248bc9799a707e36d60,2025-01-01,2025-01-01 16:57:15.062114990
1,page_view,quiz_fit,040e1e30c9ed4248bc9799a707e36d60,2025-01-01,2025-01-01 16:57:24.243407959
2,page_view,quiz_highlight,040e1e30c9ed4248bc9799a707e36d60,2025-01-01,2025-01-01 16:57:37.877279053
3,page_view,quiz_bodyShape,040e1e30c9ed4248bc9799a707e36d60,2025-01-01,2025-01-01 16:57:52.290892090
4,page_view,quiz_eyes,040e1e30c9ed4248bc9799a707e36d60,2025-01-01,2025-01-01 16:58:58.868198975


### Mapeo de secciones

In [2]:
df['section'].unique()

array(['quiz_work', 'quiz_fit', 'quiz_highlight', 'quiz_bodyShape',
       'quiz_eyes', 'quiz_hair', 'quiz_sizes', 'quiz_measurements',
       'quiz_focus', 'quiz_adventurous', 'quiz_styles', 'quiz_prices',
       'quiz_avoid', 'quiz_footwear', 'quiz_you', 'quiz_photos',
       'quiz_social', 'quiz_signUp', 'quiz_leisure'], dtype=object)

In [3]:
step_order_map = {
    "quiz_leisure": 1, #obligatoria
    "quiz_work": 2, #obligatoria
    "quiz_fit": 3,
    "quiz_highlight": 4,
    "quiz_bodyShape": 5, #obligatoria
    "quiz_eyes": 6,
    "quiz_hair": 7,
    "quiz_sizes": 8, #obligatoria
    "quiz_measurements": 9,
    "quiz_focus": 10,
    "quiz_adventurous": 11,
    "quiz_styles": 12,
    "quiz_prices": 13,
    "quiz_avoid": 14,
    "quiz_footwear": 15,
    "quiz_you": 16,
    "quiz_photos": 17,
    "quiz_social": 18,
    "quiz_signUp": 19,
}

# fakta brands, colors and patters, register por ejemplo
# AQUI FALLA ALGO PORQUE LO DE PHOTOS, SOCIAL NO LO HE VISTO

df["step_order"] = df["section"].map(step_order_map) # asignar número de paso a cada fila
df = df[df["step_order"].notna()]


In [4]:
df = df.drop_duplicates(subset=["user_id", "section", "event_timestamp_"])

## CONSTRUIR EL FUNNEL

### ¿Cuántas usaurias llegan a cada paso?
Queremos para cada paso del funnel, cuántos usuarios llegan al menos una vez a esa sección.

In [5]:
# primer timestamp en el que cada user ve cada sección
first_views = (
    df.groupby(["user_id", "section"], as_index=False).agg(
          first_time=("event_timestamp_", "min"),
      )
)

# añadimos el step_order
first_views["step_order"] = first_views["section"].map(step_order_map)


In [6]:
# número de usuarios únicos que llegan a cada paso
funnel = (
    first_views
    .groupby("step_order")
    .agg(users=("user_id", "nunique"))
    .reset_index()
    .sort_values("step_order")
)

# recuperar el nombre de la sección
inv_step_order_map = {v: k for k, v in step_order_map.items()}
funnel["section"] = funnel["step_order"].map(inv_step_order_map)

# usuarios totales que empiezan el funnel (al menos 1 sección del quiz)
total_users = first_views["user_id"].nunique() # hay en total 41 usuarios únicos que han pasado por alguna pantalla del quiz

# % de usuarios desde el inicio
funnel["from_start_rate"] = funnel["users"] / total_users

# % de usuarios desde el paso anterior
funnel["from_prev_rate"] = funnel["users"] / funnel["users"].shift(1)
print(funnel)

    step_order  users            section  from_start_rate  from_prev_rate
0            1     16       quiz_leisure         0.390244             NaN
1            2     37          quiz_work         0.902439        2.312500
2            3     37           quiz_fit         0.902439        1.000000
3            4     36     quiz_highlight         0.878049        0.972973
4            5     36     quiz_bodyShape         0.878049        1.000000
5            6     36          quiz_eyes         0.878049        1.000000
6            7     36          quiz_hair         0.878049        1.000000
7            8     36         quiz_sizes         0.878049        1.000000
8            9     36  quiz_measurements         0.878049        1.000000
9           10     36         quiz_focus         0.878049        1.000000
10          11     36   quiz_adventurous         0.878049        1.000000
11          12     36        quiz_styles         0.878049        1.000000
12          13     36        quiz_pric

In [7]:
df["section"].value_counts()

section
quiz_styles          47
quiz_hair            46
quiz_work            45
quiz_sizes           44
quiz_prices          44
quiz_fit             44
quiz_adventurous     44
quiz_bodyShape       44
quiz_highlight       44
quiz_measurements    43
quiz_eyes            43
quiz_signUp          43
quiz_avoid           42
quiz_you             42
quiz_photos          42
quiz_social          42
quiz_focus           41
quiz_footwear        30
quiz_leisure         24
Name: count, dtype: int64

En Lookiero, si una usuaria ya ha rellenado parte del quiz antes, cuando vuelve puede saltar directamente a la sección donde lo dejó. Eso hace que en los datos existan usuarios que empiezan directamente en quiz_work o quiz_fit, sin pasar por quiz_leisure.

Por eso quiz_work tiene más usuarios únicos (37) que `quiz_leisure (16).

quiz_footwear puede ser opcional, o solo se muestra a ciertas usuarias (por ejemplo, si marcan que les interesa el calzado). Hya una bajada de 36 a 26 usuarios

In [8]:
# con que pantalla empieza cada usuaria
first_section_per_user = df.sort_values("event_timestamp_").groupby("user_id").first()["section"]
first_section_per_user.value_counts()

section
quiz_work       27
quiz_leisure    13
quiz_signUp      1
Name: count, dtype: int64

Quiz word parece ser el inicio real del quiz para la mayoria.
Hay una usauria que probablemnete ya tenía sesión iniciada y fue redirigida directamente a la parte final.

In [9]:
funnel.sort_values("from_prev_rate").head(5) # donde caae mucho, son los puntos donde la genet abandona


,step_order,users,section,from_start_rate,from_prev_rate
14,15,26,quiz_footwear,0.634146,0.722222
3,4,36,quiz_highlight,0.878049,0.972973
9,10,36,quiz_focus,0.878049,1.000000
16,17,36,quiz_photos,0.878049,1.000000
13,14,36,quiz_avoid,0.878049,1.000000


## ANALIZAR TIEMPOS POR PASO

In [10]:
# ordenamos por usuario y step_order
fv_sorted = first_views.sort_values(["user_id", "step_order"])

# calculamos el tiempo hasta el siguiente paso
fv_sorted["next_time"] = fv_sorted.groupby("user_id")["first_time"].shift(-1)
fv_sorted["time_to_next"] = (
    fv_sorted["next_time"] - fv_sorted["first_time"]
).dt.total_seconds()


In [11]:
time_stats = (
    fv_sorted
    .groupby("step_order")
    .agg(median_time_to_next=("time_to_next", "median"))
    .reset_index()
)
time_stats["section"] = time_stats["step_order"].map(inv_step_order_map)


In [12]:
time_stats
# si un paso tiene muy alto tiempo y además muchas salidas después, probablemente es un pain point de UX (difícil de rellenar, genera dudas, etc.).

,step_order,median_time_to_next,section
0,1,8.864713,quiz_leisure
1,2,9.816954,quiz_work
2,3,18.482816,quiz_fit
3,4,20.856579,quiz_highlight
4,5,34.177424,quiz_bodyShape
5,6,11.363381,quiz_eyes
6,7,8.616449,quiz_hair
7,8,31.851825,quiz_sizes
8,9,13.657588,quiz_measurements
9,10,19.151518,quiz_focus


Pasos “ligeros” (pocas opciones, decisión sencilla) → ~8–12 s
Ej: quiz_leisure, quiz_work, quiz_hair, quiz_eyes.

Pasos “pesados / complejos” → tiempos altos:
- quiz_bodyShape ~34 s
- quiz_sizes ~32 s
- quiz_adventurous ~27 s
- quiz_styles ~66 s (clarísimo “cuello de botella”)
- quiz_photos ~54 s

Interpretación UX: quiz_styles y quiz_photos parecen ser pantallas donde la gente se entretiene más, duda más o se bloquea:

Muchas opciones, texto largo, decisiones poco claras, o simplemente un paso donde la usuaria se lo toma con calma.

## Detectar backtracking o bucles (usuario se lía)
Mirar si las usuarias van hacia atrás en el cuestionario (por ejemplo, dejan quiz_styles y vuelven a quiz_fit):

In [13]:
df_sorted = df.sort_values(["user_id", "event_timestamp_"]).copy()
df_sorted["prev_step"] = df_sorted.groupby("user_id")["step_order"].shift(1)

# cuando el step actual es menor que el anterior → ha ido hacia atrás
backtracks = df_sorted[df_sorted["step_order"] < df_sorted["prev_step"]]

# cuántos backtracks hay por sección
backtrack_stats = (
    backtracks
    .groupby("section")
    .agg(n_backtracks=("user_id", "nunique"))
    .reset_index()
    .sort_values("n_backtracks", ascending=False)
)

backtrack_stats # secciones con muchos backtracks → pueden indicar que la usuaria no entiende bien la pregunta o que necesita revisar sus respuestas.

,section,n_backtracks
9,quiz_leisure,5
7,quiz_hair,5
16,quiz_work,4
15,quiz_styles,4
14,quiz_social,4
12,quiz_prices,3
8,quiz_highlight,3
0,quiz_adventurous,3
4,quiz_fit,3
2,quiz_bodyShape,3


In [14]:
backtrack_stats.to_csv("Datos/Transformados/backtrack_stats.csv")
time_stats.to_csv("Datos/Transformados/time_stats.csv")